In [3]:
import pandas as pd
import numpy as np
import gzip
import math
import matplotlib.pyplot as plt
import seaborn as sns
import string
from collections import defaultdict
import regex as re
from dateutil.parser import parse

In [4]:
from fashionEDA import fashionCleaner

In [5]:
fc = fashionCleaner()

Scrapping Data, Null Entry: 1
Scrapping Data, Null Entry: 2
Scrapping Data, Null Entry: 3
Scrapping Data, Null Entry: 4
Scrapping Data, Null Entry: 5
Scrapping Data, Null Entry: 6
Scrapping Data, Null Entry: 7
Scrapping Data, Null Entry: 8
Scrapping Data, Null Entry: 9
Scrapping Data, Null Entry: 10
Scrapping Data, Null Entry: 11
Scrapping Data, Null Entry: 12
Scrapping Data, Null Entry: 13
Scrapping Data, Null Entry: 14
Scrapping Data, Null Entry: 15
Scrapping Data, Null Entry: 16
Scrapping Data, Null Entry: 17
Scrapping Data, Null Entry: 18
Scrapping Data, Null Entry: 19
Scrapping Data, Null Entry: 20
Scrapping Data, Null Entry: 21
Scrapping Data, Null Entry: 22
Scrapping Data, Null Entry: 23
Scrapping Data, Null Entry: 24
Scrapping Data, Null Entry: 25
Scrapping Data, Null Entry: 26
Scrapping Data, Null Entry: 27
Scrapping Data, Null Entry: 28
Scrapping Data, Null Entry: 29
Scrapping Data, Null Entry: 30
Scrapping Data, Null Entry: 31
Scrapping Data, Null Entry: 32
Scrapping Data, N

In [6]:
fashion_df = fc.getData()
fashion_df.head()

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date,bust,cup
0,fit,420272,34d,2260466,137,10,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,68,14,28,2016-04-20,34,d
1,fit,273551,34b,153475,132,10,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,66,12,36,2013-06-18,34,b
2,fit,909926,34c,126335,135,8,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,65,8,34,2014-02-12,34,c
3,fit,151944,34b,616682,145,10,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,69,12,27,2016-09-26,34,b
4,fit,734848,32b,364092,138,8,date,Didn't actually wear it. It fit perfectly. The...,athletic,Traditional with a touch a sass,dress,68,8,45,2016-04-30,32,b


In [8]:
#tf-idf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, precision_recall_fscore_support

In [67]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [68]:
# Download the NLTK English tokenizer and the stopwords of all languages
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\brian\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\brian\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [ ]:
punctuation = set(string.punctuation)

clean_words = [] 
for i in range(len(words)):
    clean_words.append(''.join([c for c in words[i].lower() if not c in punctuation]))

In [83]:
words = " ".join(list(fashion_df['review_text'].values))
summary = " ".join(list(fashion_df['review_summary'].values))

In [84]:
words = words.split()
summary = summary.split()

In [198]:
vectorizer = CountVectorizer(analyzer='word',
                        max_features = 10000,
                        tokenizer=word_tokenize,
                        stop_words=stopwords.words('english'))

In [335]:
#tfidf
tfidf = TfidfVectorizer(sublinear_tf=True,
                        analyzer='word',
                        max_features=1500,
                        tokenizer=word_tokenize)

In [336]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [337]:
mod = LinearRegression()

In [338]:
as_is = ['weight', 'size', 'height', 'age', 'rating']
ohe = ['rented for', 'body type', 'category']
text_vec = ['review_summary']

In [339]:
text_transformer = make_column_transformer((tfidf, 'review_summary'), remainder = 'passthrough')
#so we can use vectorizer on the review_summary

In [340]:
ct = ColumnTransformer(
    transformers = [("as_is", FunctionTransformer(lambda x: x), as_is),
    ("ohe", OneHotEncoder(handle_unknown = 'ignore'), ohe),
    ("vec", text_transformer, text_vec)]
)

In [341]:
pl = Pipeline(steps = [('transform', ct), ('linear_regression', mod)])

In [342]:
train, test = train_test_split(fashion_df, test_size = 0.2, random_state = 200)

In [343]:
train_X, train_Y = train[['weight', 'size', 'height', 'age', 'rating', 'rented for', 'body type', 'category', 'review_summary']], train['bust']
test_X, test_Y = test[['weight', 'size', 'height', 'age', 'rating', 'rented for', 'body type', 'category', 'review_summary']], test['bust']

In [344]:
pl.fit(train_X, train_Y)

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('as_is',
                                                  FunctionTransformer(func=<function <lambda> at 0x00000272E9E4BC10>),
                                                  ['weight', 'size', 'height',
                                                   'age', 'rating']),
                                                 ('ohe',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['rented for', 'body type',
                                                   'category']),
                                                 ('vec',
                                                  ColumnTransformer(remainder='passthrough',
                                                                    transformers=[('tfidfvectorizer',
                                                                                   TfidfVectorizer(ma

In [345]:
preds = pl.predict(test_X)

In [346]:
preds = [round(pred) for pred in preds]

In [347]:
sum([1 if p == a else 0 for p, a in zip(preds, test_Y)])/len(preds)

0.3117122656009837

In [348]:
mean_squared_error(preds, test_Y)

1.6218533319670732